# Exploring Geo-Data

## Loading Data

In this section we download the data from a remote server and save it to a working directory on our computer.

### Setup

In [ ]:
from pathlib import Path

src_dir = Path.home() / Path('work/data/clean')

if not src_dir.exists():
    src_dir.mkdir(parents=True)
    print(f"Creating a 'clean' directory for data in {src_dir}.")

Below we are setting up some basic *variables* that we'll use to manage the download porcess. The data is all accessible from a CASA server called 'Orca' (`base_url`) and then we have a list of the data and geodata that we will need to download. Note that, when approaching a problem with code, we can be 'lazy' and not even specify the file extension (`.parquet`) because we can *add* then when downloading each file.

In [ ]:
base_url = 'https://orca.casa.ucl.ac.uk/~jreades/jaipur'

geofiles = [
    'Jaipur_Boundary', 'Jaipur_Wards', 'northern-zone-lines', 'northern-zone-multilines', 'northern-zone-multipolygons', 
    'northern-zone-other', 'northern-zone-points', 'India_Country_Boundary', 'India_State_Boundary'
]

print(f"The geo-data files are: {', '.join(geofiles)}")

### Download

Notice the `exists()` part -- we are checking for the existence of a file locally *before* we try to download it!

In [ ]:
# urlretrieve is a function (provided by Python) for 
# downloading a file from a URL and saving it locally
from urllib.request import urlretrieve

# For each geo-data file
for f in geofiles:
    print(f"Retrieving {f} table.")
    save_path = Path(src_dir / f"{f}.geoparquet")
    if save_path.exists():
        print("\tAlready downloaded this data...")
    else: 
        print("\tDownloading...")
        urlretrieve(f"{base_url}/{f}.geoparquet", save_path)

## Geo-Data

All *geo*-data needs to be read in using *geo*-pandas, which we import using an 'alias' (`gpd`) to save time.

In [ ]:
import geopandas as gpd # We only need to do this *once*

## A Base Map

This next group of commands isn't very friendly to the orca server (especially if you *keep* running this code!), but it is useful for demonstrating that you can even read a file on another server when it's in the parquet format. We literally just 'request' the file and geopandas can load it dynamically. 

In [ ]:
import requests, io
result = requests.get(f"{base_url}/India_Country_Boundary.geoparquet")
india = gpd.read_parquet(io.BytesIO(result.content))
india.plot()

In [ ]:
istates = gpd.read_parquet(src_dir / 'India_State_Boundary.geoparquet')
istates.plot()

In [ ]:
istates.head()

In [ ]:
raja = istates[istates.State_Name=='Rajasthan']
raja.plot()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib_map_utils.core.north_arrow import NorthArrow, north_arrow

f, ax = plt.subplots(1,1,figsize=(7,7))
india.plot(ax=ax)
raja.plot(color='red', ax=ax)
plt.text(raja.centroid.x.iloc[0], raja.centroid.y.iloc[0], 'Rajasthan', horizontalalignment='center', verticalalignment='center')
scalebar = ScaleBar(1)
ax.add_artist(scalebar)
north_arrow(
    ax, location="lower left", rotation={"crs": india.crs, "reference": "center"}
)

## Exploring Jaipur

Let's start by exploring the boundary file for Jaipur, which we downloaded from Orca and are now going to read in.

In [ ]:
jp = gpd.read_parquet(src_dir / 'Jaipur_Boundary.geoparquet')
# What does it contain?
print(f"`jp` has shape: {jp.shape}")

That `shape` output isn't very intelligible; fortunately, we with a small file we can also *show* what it contains quite easily.

In [ ]:
jp

In [ ]:
jp.Id # `loc[<index identifier>]` is how we access an individual row

In [ ]:
jp.geometry

Notice the `0` on the left: that's the same `0` shown in bold when we just run `jp`. This is an **index**, which is a kind of row identifier. Technically, indexes can be labels or numbers, but here we just have the row number.

So, that's all well and good, but what does it look like? Let's find out!

In [ ]:
jp.plot();

In [ ]:
jp.plot(color='red');

In [ ]:
osml = gpd.read_parquet(src_dir / 'northern-zone-lines.geoparquet')
osml.plot();

To show these together we need to combine them into one plot. The way we do this in Python is as follows:

In [ ]:
import matplotlib.pyplot as plt

f, axis = plt.subplots(1,1,figsize=(7,5))
axis.set_title("Broken Map")
jp.plot(color='red', ax=axis)
osml.plot(color='orange', ax=axis);

<div class="alert alert-block alert-warning">

### &#9888; Warning

That 'map' doesn't look right, does it? Can the *planners* tell us why? Below is a hint...

</div>

In [ ]:
print(f"The Jaipur CRS (map projection) is: {jp.crs.name}")
print()
print(f"The OSM Lines CRS (map projection) is: {osml.crs.name}")

### A Quick Map

In [ ]:
f, axis = plt.subplots(1,1,figsize=(7,5))
axis.set_title("Slightly better map")
osml.to_crs(jp.crs).plot(color='orange', ax=axis)
jp.plot(color='red', ax=axis);

We can use the bounding box for Jaipur to set up the x and y 'limits' of the map.

In [ ]:
print(jp.bounds)

In [ ]:
f, axis = plt.subplots(1,1,figsize=(7,5))
osml.to_crs(jp.crs).plot(color='orange', ax=axis)
jp.plot(color='red', ax=axis)
axis.set_xlim([jp.bounds.minx[0], jp.bounds.maxx[0]])
axis.set_ylim([jp.bounds.miny[0], jp.bounds.maxy[0]])

### Buffering a Boundary

But it feels quite awkward cropping the map to *exactly* the edges of the city. Python makes it easy to buffer the border and use *that* instead.

In [ ]:
jpb = jp.buffer(5000)
print(jpb.bounds)

In [ ]:
f, axis = plt.subplots(1,1,figsize=(7,5))
osml.to_crs(jp.crs).plot(color='orange', ax=axis)
jp.plot(color='red', ax=axis)

# And now set the limits of the map using the buffer
axis.set_xlim([jpb.bounds.minx[0], jpb.bounds.maxx[0]])
axis.set_ylim([jpb.bounds.miny[0], jpb.bounds.maxy[0]])

## Exploring OSM Data
What *are* all those orange lines? They make it almost impossible to read the map!

Using `head` we can look at the first few records from the data.

In [ ]:
osml.head()

In [ ]:
osml.highway.unique()

In [ ]:
highways  = osml[osml.highway.isin(['trunk','primary','secondary'])].to_crs(jp.crs)
waterways = osml[~osml.waterway.isna()].to_crs(jp.crs)

In [ ]:
print(f"Highways has shape {highways.shape}")
print(f"Waterways has shape {waterways.shape}")

## Pulling it All Together

Here is what we want to achieve... 

In [ ]:
f, axis = plt.subplots(1,1,figsize=(7,5))
highways.plot(color="grey", ax=axis)
waterways.plot(color="blue", ax=axis)
jp.plot(color='red', ax=axis)
axis.set_xlim([jpb.bounds.minx[0], jpb.bounds.maxx[0]])
axis.set_ylim([jpb.bounds.miny[0], jpb.bounds.maxy[0]])

Read more about [colourmaps](https://matplotlib.org/stable/users/explain/colors/colormaps.html) in the documentation.

And here is how to specify a colour [using hexadecimal](https://www.google.com/search?q=hex+color+picker).

In [ ]:
f, axis = plt.subplots(1,1,figsize=(7,5))
highways.plot(column='highway', cmap='copper', linewidth=0.5, ax=axis)
waterways.plot(color="#7bb1e0", ax=axis)
jp.plot(color='red', linewidth=3, ax=axis)
axis.set_xlim([jpb.bounds.minx[0], jpb.bounds.maxx[0]])
axis.set_ylim([jpb.bounds.miny[0], jpb.bounds.maxy[0]]);

We can go a lot further than this (as you will have seen from my talk) but I thought that this was more than enough to be starting with!

### Working Out What to Show

Below I just wanted to leave in my working process for puzzling through what was in a data file with which I am *completely* unfamiliar! You'll see that I start my trying to look at the overall distribution. This is a crude approach, but I don't honestly know what I'm looking for here! So I slowly work my way from distributions to things that I can put on a map...

In [ ]:
import geopandas as gpd

#### OSM Lines File

In [ ]:
osml.columns.to_list()

In [ ]:
osml.highway.value_counts()

In [ ]:
osml_resi  = osml[osml.highway=='residential']
osml_main  = osml[osml.highway.isin(['motorway','trunk_link','primary_link','secondary_link','tertiary_link'])]
osml_minor = osml[osml.highway.isin(['footway','pedestrian','track'])]

#### OSM Points File

In [ ]:
osmp.columns.to_list()

In [ ]:
osmp.other_tags.value_counts()

In [ ]:
osmp_bus = osmp[osmp.other_tags=='"bus"=>"yes","public_transport"=>"platform"']

In [ ]:
wards  = gpd.read_parquet(src_dir / 'Jaipur_Wards.geoparquet')
osmp = gpd.read_parquet(src_dir / 'northern-zone-points.geoparquet')
osml = gpd.read_parquet(src_dir / 'northern-zone-lines.geoparquet')

f, ax = plt.subplots(1,1,figsize=(8,5))
wards.plot(ax=ax, edgecolor='blue', color="None", linewidth=0.25, zorder=3)
osml_main.to_crs(wards.crs).plot(ax=ax, color='red', linewidth=1.5, zorder=1)
osml_resi.to_crs(wards.crs).plot(ax=ax, color='orange', linewidth=1., zorder=1)
osml_minor.to_crs(wards.crs).plot(ax=ax, color='green', linewidth=0.5, zorder=1)
osmp_bus.to_crs(wards.crs).plot(ax=ax, color='green', zorder=2)
ax.set_ylim([2975000, 2980000])
ax.set_xlim([575000, 580000])

## Raster Data

I wanted to also demonstrate how to add raster data but the 'open' India Geo-Platform of ISRO [requires a government-issue login](https://bhuvan-app3.nrsc.gov.in/data/download/index.php). Note the different definition of 'open' here!

In [ ]:
import rasterio 
from rasterio.plot import show
from pathlib import Path
import matplotlib.pyplot as plt

# I had issues with processing the raster when using
# the full-size DEM, so I downsamples it to 25% of its
# original size. The issue was lack of memory (RAM) -- 
# the podman machine with 4GB of RAM just didn't have 
# enough to handle the DEM.
src = Path( Path.home() / 'work/data/src/Jaipur_District_DEM_downsampled/DEM_Jaipur_District.tif' )

f, ax = plt.subplots(1,1,figsize=(8,5))
with rasterio.open(src) as reader:
    dem = reader.read(1)
    show(reader, ax=ax, cmap='terrain', transform=reader.transform, title='Digital Elevation Model')
    plt.title('Digital Elevation Model')

In [ ]:
import osgeo.gdal as gdal
import numpy as np
import subprocess
import matplotlib.pyplot as plt
from pathlib import Path

gdal.UseExceptions()
cmd = "gdaldem slope " + str(src.resolve()) + " slope.tif -compute_edges"
subprocess.call(cmd.split())
slope = Path( Path.home() / 'work/practicals/slope.tif' )

In [ ]:
slp = gdal.Open(slope)
slpArray = slp.GetRasterBand(1).ReadAsArray()

mask = ((slpArray < 0) | (slpArray > 50))
masked_data = np.ma.masked_array(slpArray, mask)

In [ ]:
slpArray[ slpArray < -5000 ] = np.nan
slpArray[ slpArray > 75 ] = np.nan
masked_data = np.ma.masked_invalid(slpArray)

In [ ]:
plt.imshow(masked_data, cmap='viridis')
plt.colorbar()